# RAG Tools: Database Setup

## 1. Introduction

In this notebook, we'll set up Docker containers for our PostgreSQL database with pgvector extension and Neo4j graph database. We'll ensure both databases have accessible browser interfaces for easy management.

## 2. Databases - PgVector and Neo4j

"We're using a combination of PgVector (PostgreSQL with vector extensions) and Neo4j to create a powerful RAG (Retrieval-Augmented Generation) system. PgVector allows us to store and efficiently query high-dimensional vectors, which are crucial for embedding-based search and similarity comparisons in machine learning applications.
Neo4j, being a graph database, excels at representing and querying complex relationships between entities. By combining these two databases, we can create a system that not only understands semantic similarity (through vector embeddings) but also captures and leverages the intricate relationships between different pieces of information.
This combination is particularly powerful for tasks like code analysis, where we need to understand both the content of code (which can be embedded into vectors) and the relationships between different code elements (which can be represented as a graph).

## 3. Environment Configuration

First, let's set up our environment variables. We'll create a file named `.env` in the `config/` directory:

The .env file plays a crucial role in our framework as a centralized repository for environment-specific configuration variables. As we progress through our project, we'll continually add new variables to this file, allowing us to easily manage and update our configuration settings without modifying our code. The .env file stores sensitive information like database credentials, API keys, and other configuration parameters that may vary between development, testing, and production environments. To leverage these variables effectively, we've created the config_utils tool, which acts as an interface between our application and the .env file. The Config class within config_utils uses the python-dotenv library to load variables from the .env file, making them accessible throughout our application. This approach offers several advantages: it enhances security by keeping sensitive information out of our codebase, promotes flexibility by allowing easy configuration changes without code modifications, and improves maintainability by centralizing our configuration management. As we add new components or services to our framework, we'll update both the .env file and the Config class in config_utils, ensuring that our entire application remains in sync with our latest configuration needs.

```
# PostgreSQL Configuration
POSTGRES_DB=ragtools_db
POSTGRES_USER=ragtools_user
POSTGRES_PASSWORD=secure_postgres_password
POSTGRES_HOST=localhost
POSTGRES_PORT=5432

# Neo4j Configuration
NEO4J_AUTH=neo4j/secure_neo4j_password
NEO4J_HOST=localhost
NEO4J_HTTP_PORT=7474
NEO4J_BOLT_PORT=7687

# Docker Configuration
POSTGRES_CONTAINER_NAME=ragtools_postgres
NEO4J_CONTAINER_NAME=ragtools_neo4j
DOCKER_NETWORK_NAME=ragtools_network
```

Run the next code-block to create the file in the correct directory.
 
**IMPORTANT**: REMEMBER TO UPDATE THE .ENV FILE AFTER IT IS CREATED WITH THE USERNAME AND PASSWORDS AS NEEDED.

In [3]:
import os

# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the config directory
config_dir = os.path.join(current_dir, '..', 'config')

# Ensure the config directory exists
os.makedirs(config_dir, exist_ok=True)

# Construct the full path for the .env file
env_file_path = os.path.join(config_dir, '.env')

# Content of the .env file
env_content = """
# PostgreSQL Configuration
POSTGRES_DB=ragtools_db
POSTGRES_USER=ragtools_user
POSTGRES_PASSWORD=secure_postgres_password
POSTGRES_HOST=localhost
POSTGRES_PORT=5432

# Neo4j Configuration
NEO4J_AUTH=neo4j/secure_neo4j_password
NEO4J_HOST=localhost
NEO4J_HTTP_PORT=7474
NEO4J_BOLT_PORT=7687

# Docker Configuration
POSTGRES_CONTAINER_NAME=ragtools_postgres
NEO4J_CONTAINER_NAME=ragtools_neo4j
DOCKER_NETWORK_NAME=ragtools_network
"""

# Write the content to the .env file
with open(env_file_path, 'w') as f:
    f.write(env_content)

print(f".env file created at: {env_file_path}")


.env file created at: /home/todd/ML-Lab/git/R3d91lls-LLM-Tools/RAG_tools/notebooks/../config/.env


## 4. Docker Compose Configuration

{discuss this docker-compose.yml file. why did we choose the images we chose also discuss updating the .env file that was created in hte previous step before proceeding with the next step.}
```yaml
version: '3.8'

services:
  postgres:
    image: ankane/pgvector
    container_name: ${POSTGRES_CONTAINER_NAME}
    environment:
      POSTGRES_DB: ${POSTGRES_DB}
      POSTGRES_USER: ${POSTGRES_USER}
      POSTGRES_PASSWORD: ${POSTGRES_PASSWORD}
    ports:
      - "${POSTGRES_PORT}:5432"
    volumes:
      - ./db_data/postgres:/var/lib/postgresql/data
    networks:
      - ragtools_network

  neo4j:
    image: neo4j:latest
    container_name: ${NEO4J_CONTAINER_NAME}
    environment:
      NEO4J_AUTH: ${NEO4J_AUTH}
    ports:
      - "${NEO4J_HTTP_PORT}:7474"
      - "${NEO4J_BOLT_PORT}:7687"
    volumes:
      - ./db_data/neo4j:/data
    networks:
      - ragtools_network

networks:
  ragtools_network:
    name: ${DOCKER_NETWORK_NAME}

volumes:
  postgres_data:
  neo4j_data:
```

Now, let's create our `docker-compose.yml` file.

In [4]:
import os
import yaml

# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the config directory
config_dir = os.path.join(current_dir, '..', 'config')

# Ensure the config directory exists
os.makedirs(config_dir, exist_ok=True)

# Construct the full path for the docker-compose.yml file
docker_compose_path = os.path.join(config_dir, 'docker-compose.yml')

# Docker Compose configuration
docker_compose_config = {
    'version': '3.8',
    'services': {
        'postgres': {
            'image': 'ankane/pgvector',
            'container_name': '${POSTGRES_CONTAINER_NAME}',
            'environment': {
                'POSTGRES_DB': '${POSTGRES_DB}',
                'POSTGRES_USER': '${POSTGRES_USER}',
                'POSTGRES_PASSWORD': '${POSTGRES_PASSWORD}'
            },
            'ports': ['${POSTGRES_PORT}:5432'],
            'volumes': ['./db_data/postgres:/var/lib/postgresql/data'],
            'networks': ['ragtools_network']
        },
        'neo4j': {
            'image': 'neo4j:latest',
            'container_name': '${NEO4J_CONTAINER_NAME}',
            'environment': {
                'NEO4J_AUTH': '${NEO4J_AUTH}'
            },
            'ports': [
                '${NEO4J_HTTP_PORT}:7474',
                '${NEO4J_BOLT_PORT}:7687'
            ],
            'volumes': ['./db_data/neo4j:/data'],
            'networks': ['ragtools_network']
        }
    },
    'networks': {
        'ragtools_network': {
            'name': '${DOCKER_NETWORK_NAME}'
        }
    },
    'volumes': {
        'postgres_data': None,
        'neo4j_data': None
    }
}

# Write the Docker Compose configuration to the file
with open(docker_compose_path, 'w') as f:
    yaml.dump(docker_compose_config, f, default_flow_style=False)

print(f"Updated docker-compose.yml file created at: {docker_compose_path}")


Updated docker-compose.yml file created at: /home/todd/ML-Lab/git/R3d91lls-LLM-Tools/RAG_tools/notebooks/../config/docker-compose.yml


## 5 Configuration Utility

To manage our configuration variables more efficiently, we'll create a `Config` class in a file named `config_utils.py`. This class will load environment variables from our `.env` file and provide easy access to these configurations throughout our project.

Here's the content for the `config_utils.py` file:

```python
import os
from dotenv import load_dotenv

class Config:
    def __init__(self):
        load_dotenv()
        
        # Database configurations
        self.POSTGRES_DB = os.getenv('POSTGRES_DB')
        self.POSTGRES_USER = os.getenv('POSTGRES_USER')
        self.POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
        self.POSTGRES_HOST = os.getenv('POSTGRES_HOST')
        self.POSTGRES_PORT = os.getenv('POSTGRES_PORT')
        
        self.NEO4J_AUTH = os.getenv('NEO4J_AUTH')
        self.NEO4J_HOST = os.getenv('NEO4J_HOST')
        self.NEO4J_HTTP_PORT = os.getenv('NEO4J_HTTP_PORT')
        self.NEO4J_BOLT_PORT = os.getenv('NEO4J_BOLT_PORT')
        
        # pgAdmin configurations
        self.PGADMIN_PORT = os.getenv('PGADMIN_PORT')
        self.PGADMIN_DEFAULT_EMAIL = os.getenv('PGADMIN_DEFAULT_EMAIL')
        self.PGADMIN_DEFAULT_PASSWORD = os.getenv('PGADMIN_DEFAULT_PASSWORD')
        
        # Docker configurations
        self.POSTGRES_CONTAINER_NAME = os.getenv('POSTGRES_CONTAINER_NAME')
        self.NEO4J_CONTAINER_NAME = os.getenv('NEO4J_CONTAINER_NAME')
        self.PGADMIN_CONTAINER_NAME = os.getenv('PGADMIN_CONTAINER_NAME')
        self.DOCKER_NETWORK_NAME = os.getenv('DOCKER_NETWORK_NAME')

    def get_postgres_connection_params(self):
        return {
            "dbname": self.POSTGRES_DB,
            "user": self.POSTGRES_USER,
            "password": self.POSTGRES_PASSWORD,
            "host": self.POSTGRES_HOST,
            "port": self.POSTGRES_PORT
        }

        def print_all_attributes(self):
        print("All Config attributes:")
        for attr, value in self.__dict__.items():
            print(f"{attr}: {value}")

    def get_neo4j_connection_params(self):
        return {
            "uri": f"bolt://{self.NEO4J_HOST}:{self.NEO4J_BOLT_PORT}",
            "auth": tuple(self.NEO4J_AUTH.split('/'))
        }
```

This `Config` class provides a centralized way to access our configuration variables. It loads the environment variables from the `.env` file and stores them as attributes. It also includes methods to easily retrieve connection parameters for our databases. 

Run the next code-block to create the DockerComposeManager class file in the correct directory

In [5]:
import os

# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the src/utils directory
utils_dir = os.path.join(current_dir, '..', 'src', 'utils')

# Ensure the utils directory exists
os.makedirs(utils_dir, exist_ok=True)

# Construct the full path for the config_utils.py file
config_utils_path = os.path.join(utils_dir, 'config_utils.py')

# Updated content of the config_utils.py file
config_utils_content = '''
import os
from dotenv import load_dotenv

class Config:
    def __init__(self):
        load_dotenv()
        
        # Database configurations
        self.POSTGRES_DB = os.getenv('POSTGRES_DB')
        self.POSTGRES_USER = os.getenv('POSTGRES_USER')
        self.POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
        self.POSTGRES_HOST = os.getenv('POSTGRES_HOST')
        self.POSTGRES_PORT = os.getenv('POSTGRES_PORT')
        
        self.NEO4J_AUTH = os.getenv('NEO4J_AUTH')
        self.NEO4J_HOST = os.getenv('NEO4J_HOST')
        self.NEO4J_HTTP_PORT = os.getenv('NEO4J_HTTP_PORT')
        self.NEO4J_BOLT_PORT = os.getenv('NEO4J_BOLT_PORT')
        
        # pgAdmin configurations
        self.PGADMIN_PORT = os.getenv('PGADMIN_PORT')
        self.PGADMIN_DEFAULT_EMAIL = os.getenv('PGADMIN_DEFAULT_EMAIL')
        self.PGADMIN_DEFAULT_PASSWORD = os.getenv('PGADMIN_DEFAULT_PASSWORD')
        
        # Docker configurations
        self.POSTGRES_CONTAINER_NAME = os.getenv('POSTGRES_CONTAINER_NAME')
        self.NEO4J_CONTAINER_NAME = os.getenv('NEO4J_CONTAINER_NAME')
        self.PGADMIN_CONTAINER_NAME = os.getenv('PGADMIN_CONTAINER_NAME')
        self.DOCKER_NETWORK_NAME = os.getenv('DOCKER_NETWORK_NAME')

    def get_postgres_connection_params(self):
        return {
            "dbname": self.POSTGRES_DB,
            "user": self.POSTGRES_USER,
            "password": self.POSTGRES_PASSWORD,
            "host": self.POSTGRES_HOST,
            "port": self.POSTGRES_PORT
        }

    def print_all_attributes(self):
        print("All Config attributes:")
        for attr, value in self.__dict__.items():
            print(f"{attr}: {value}")

    def get_neo4j_connection_params(self):
        return {
            "uri": f"bolt://{self.NEO4J_HOST}:{self.NEO4J_BOLT_PORT}",
            "auth": tuple(self.NEO4J_AUTH.split('/'))
        }
'''

# Write the content to the config_utils.py file
with open(config_utils_path, 'w') as f:
    f.write(config_utils_content)

print(f"Updated config_utils.py file created at: {config_utils_path}")


Updated config_utils.py file created at: /home/todd/ML-Lab/git/R3d91lls-LLM-Tools/RAG_tools/notebooks/../src/utils/config_utils.py


## 6. Create DockerComposeManager

In the future we may want automation to spin up a docker environment, so lets create a python tool to help up manage that. DockerComposeManager will be that utility.

{spend a paragraph discussing this code and what it does and how to use it}

```python
import subprocess
import os
from dotenv import load_dotenv

class DockerComposeManager:
    def __init__(self, compose_file_path):
        self.compose_file_path = os.path.abspath(compose_file_path)
        load_dotenv(dotenv_path=os.path.join(os.path.dirname(self.compose_file_path), '.env'))

    def run_command(self, command):
        try:
            result = subprocess.run(
                f"docker compose -f {self.compose_file_path} {command}",
                shell=True, check=True, capture_output=True, text=True
            )
            print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"Error executing command: {e}")
            print(e.stderr)

    def start_containers(self):
        self.run_command("up -d")

    def stop_containers(self):
        self.run_command("down")

    def show_container_status(self):
        self.run_command("ps")
```

Run the next code-block to create the DockerComposeManager class file in the correct directory

In [6]:
import os

# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the src/utils directory
utils_dir = os.path.join(current_dir, '..', 'src', 'utils')

# Ensure the utils directory exists
os.makedirs(utils_dir, exist_ok=True)

# Construct the full path for the DockerComposeManager.py file
docker_compose_manager_path = os.path.join(utils_dir, 'DockerComposeManager.py')

# Content of the DockerComposeManager.py file
docker_compose_manager_content = """
import subprocess
import os
from dotenv import load_dotenv

class DockerComposeManager:
    def __init__(self, compose_file_path):
        self.compose_file_path = os.path.abspath(compose_file_path)
        load_dotenv(dotenv_path=os.path.join(os.path.dirname(self.compose_file_path), '.env'))

    def run_command(self, command):
        try:
            result = subprocess.run(
                f"docker compose -f {self.compose_file_path} {command}",
                shell=True, check=True, capture_output=True, text=True
            )
            print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"Error executing command: {e}")
            print(e.stderr)

    def start_containers(self):
        self.run_command("up -d")

    def stop_containers(self):
        self.run_command("down")

    def show_container_status(self):
        self.run_command("ps")
"""

# Write the content to the DockerComposeManager.py file
with open(docker_compose_manager_path, 'w') as f:
    f.write(docker_compose_manager_content)

print(f"DockerComposeManager.py file created at: {docker_compose_manager_path}")


DockerComposeManager.py file created at: /home/todd/ML-Lab/git/R3d91lls-LLM-Tools/RAG_tools/notebooks/../src/utils/DockerComposeManager.py


## 7 updating our directory structure for persistent databases

this will create a new  directory for use by the docker containers which will now look like this:

```
RAG_tools/
├── config/
│   ├── docker-compose.yml
│   ├── .env
├── db_data/
│   ├── neo4j
│   └── postgres
├── notebooks/
│   ├── 00_Environment_Setup.ipynb
│   ├── 01_Project_Overview_and_Architecture.ipynb
│   ├── 02_Model_Selection_and_Rationale.ipynb
├── src/
│   └── utils/
│       ├── config_utils.py
│       └── DockerComposeManager.py
└── tests/
```

Execute the next code block to create the new directories.

In [7]:
import os

# Get the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Create a directory for database persistence
db_dir = os.path.join(project_root, 'db_data')
os.makedirs(db_dir, exist_ok=True)

# Create subdirectories for each database
postgres_dir = os.path.join(db_dir, 'postgres')
neo4j_dir = os.path.join(db_dir, 'neo4j')

os.makedirs(postgres_dir, exist_ok=True)
os.makedirs(neo4j_dir, exist_ok=True)

print(f"Created database directories:")
print(f"PostgreSQL: {postgres_dir}")
print(f"Neo4j: {neo4j_dir}")


Created database directories:
PostgreSQL: /home/todd/ML-Lab/git/R3d91lls-LLM-Tools/RAG_tools/db_data/postgres
Neo4j: /home/todd/ML-Lab/git/R3d91lls-LLM-Tools/RAG_tools/db_data/neo4j


## 5. Launch Docker Containers

{spend some time about how we have created the tools and now are linking them together to get something done. also note that through this whole process we will be creating tools that we will use to build larger things}

**WARNING**: BEFORE PROCEEDING WITH THE NEXT CODE-BLOCK DO NOT FORGET TO UPDATE YOUR .env FILE.
Now that we have our `config_utility` and `DockerComposeManager`, let's use them to launch our Docker containers:

In [8]:
import os
import sys
import time
import docker

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from src.utils.DockerComposeManager import DockerComposeManager
from src.utils.config_utils import Config

# Load configuration
config = Config()

# Create an instance of DockerComposeManager
docker_manager = DockerComposeManager('../config/docker-compose.yml')

# Start the containers
print("Starting Docker containers...")
docker_manager.start_containers()

# Wait for a few seconds to allow containers to fully start
time.sleep(10)

# Check the status of the containers
print("\nChecking container status:")
docker_manager.show_container_status()

import os
import sys
import time
import docker

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from src.utils.DockerComposeManager import DockerComposeManager
from src.utils.config_utils import Config

# Load configuration
config = Config()

# Create an instance of DockerComposeManager
docker_compose_path = os.path.join(project_root, 'config', 'docker-compose.yml')
docker_manager = DockerComposeManager(docker_compose_path)

# Start the containers
print("Starting Docker containers...")
docker_manager.start_containers()

# Wait for a few seconds to allow containers to fully start
time.sleep(10)

# Check the status of the containers
print("\nChecking container status:")
docker_manager.show_container_status()

# Verify that all expected containers are running
client = docker.from_env()
expected_containers = [config.POSTGRES_CONTAINER_NAME, config.NEO4J_CONTAINER_NAME]
all_running = True

for container_name in expected_containers:
    try:
        containers = client.containers.list(filters={'name': container_name})
        if containers:
            container = containers[0]
            if container.status == 'running':
                print(f"{container_name} is running.")
            else:
                print(f"{container_name} is not running. Status: {container.status}")
                all_running = False
        else:
            print(f"{container_name} not found.")
            all_running = False
    except docker.errors.APIError as e:
        print(f"Error checking container {container_name}: {e}")
        all_running = False

if all_running:
    print("\nAll containers are running successfully!")
else:
    print("\nSome containers are not running. Please check the logs for more information.")

# Print connection information
print("\nConnection Information:")
print(f"PostgreSQL: {config.POSTGRES_HOST}:{config.POSTGRES_PORT}")
print(f"Neo4j (HTTP): {config.NEO4J_HOST}:{config.NEO4J_HTTP_PORT}")
print(f"Neo4j (Bolt): {config.NEO4J_HOST}:{config.NEO4J_BOLT_PORT}")


Starting Docker containers...


Checking container status:
NAME                IMAGE             COMMAND                  SERVICE    CREATED          STATUS          PORTS
ragtools_neo4j      neo4j:latest      "tini -g -- /startup…"   neo4j      10 seconds ago   Up 10 seconds   0.0.0.0:7474->7474/tcp, :::7474->7474/tcp, 7473/tcp, 0.0.0.0:7687->7687/tcp, :::7687->7687/tcp
ragtools_postgres   ankane/pgvector   "docker-entrypoint.s…"   postgres   10 seconds ago   Up 10 seconds   0.0.0.0:5432->5432/tcp, :::5432->5432/tcp

Starting Docker containers...


Checking container status:
NAME                IMAGE             COMMAND                  SERVICE    CREATED          STATUS          PORTS
ragtools_neo4j      neo4j:latest      "tini -g -- /startup…"   neo4j      20 seconds ago   Up 20 seconds   0.0.0.0:7474->7474/tcp, :::7474->7474/tcp, 7473/tcp, 0.0.0.0:7687->7687/tcp, :::7687->7687/tcp
ragtools_postgres   ankane/pgvector   "docker-entrypoint.s…"   postgres   20 seconds ago   Up 20 secon

## 6. Verify Container Status and Database Connections

Now that we've launched our containers, let's verify their status and test our connections to both PostgreSQL and Neo4j databases:

In [9]:
import psycopg2
from neo4j import GraphDatabase
from src.utils.config_utils import Config
import time

def verify_database_connections():
    config = Config()

    def wait_for_postgres(max_attempts=5, delay=5):
        for attempt in range(max_attempts):
            try:
                conn = psycopg2.connect(**config.get_postgres_connection_params())
                conn.close()
                print("Successfully connected to PostgreSQL")
                return True
            except psycopg2.OperationalError as e:
                print(f"Attempt {attempt + 1}/{max_attempts}: PostgreSQL is not ready yet. Retrying in {delay} seconds...")
                time.sleep(delay)
        return False

    def wait_for_neo4j(max_attempts=5, delay=5):
        for attempt in range(max_attempts):
            try:
                driver = GraphDatabase.driver(**config.get_neo4j_connection_params())
                with driver.session() as session:
                    result = session.run("RETURN 1 AS x")
                    assert result.single()['x'] == 1
                driver.close()
                print("Successfully connected to Neo4j")
                return True
            except Exception as e:
                print(f"Attempt {attempt + 1}/{max_attempts}: Neo4j is not ready yet. Error: {e}. Retrying in {delay} seconds...")
                time.sleep(delay)
        return False

    postgres_ready = wait_for_postgres()
    neo4j_ready = wait_for_neo4j()

    if postgres_ready and neo4j_ready:
        print("\nAll database connections are successful!")
    else:
        print("\nSome database connections failed. Please check your configuration and container logs.")

    if postgres_ready:
        try:
            conn = psycopg2.connect(**config.get_postgres_connection_params())
            cur = conn.cursor()
            cur.execute("SELECT * FROM pg_available_extensions WHERE name = 'vector';")
            result = cur.fetchone()
            if result:
                print("pgvector extension is available in PostgreSQL")
            else:
                print("pgvector extension is not available. Please make sure it's installed correctly.")
            cur.close()
            conn.close()
        except Exception as e:
            print(f"Error checking pgvector extension: {e}")

    print("\nConnection Information:")
    print(f"PostgreSQL: {config.POSTGRES_HOST}:{config.POSTGRES_PORT}")
    print(f"Neo4j (HTTP): http://{config.NEO4J_HOST}:{config.NEO4J_HTTP_PORT}")
    print(f"Neo4j (Bolt): bolt://{config.NEO4J_HOST}:{config.NEO4J_BOLT_PORT}")

if __name__ == "__main__":
    verify_database_connections()


Successfully connected to PostgreSQL
Successfully connected to Neo4j

All database connections are successful!
pgvector extension is available in PostgreSQL

Connection Information:
PostgreSQL: localhost:5432
Neo4j (HTTP): http://localhost:7474
Neo4j (Bolt): bolt://localhost:7687


## 7. Accessing Database Interfaces

### 7.1 pgAdmin (PostgreSQL Interface)

To access the pgAdmin interface:

1. Open your web browser and go to `http://localhost:5050`
2. Log in with the following credentials:
   - Email: admin@example.com
   - Password: secure_pgadmin_password (as set in the .env file)
3. Add a new server with the following details:
   - Host: postgres
   - Port: 5432
   - Username: ragtools_user
   - Password: secure_postgres_password (as set in the .env file)

### 7.2 Neo4j Browser

To access the Neo4j Browser interface:

1. Open your web browser and go to `http://localhost:7474`
2. Connect using the following credentials:
   - Username: neo4j
   - Password: secure_neo4j_password (as set in the .env file)

## Conclusion

We have successfully set up and verified our PostgreSQL (with pgvector) and Neo4j databases using Docker. Both databases are now accessible through their respective browser interfaces, allowing for easy management and querying.

In the next notebook, we'll start implementing our RAG system using these databases.